In [208]:
using Pipe

mutable struct Node{T}
    val::T
    next::Union{Nothing, Node{T}}
end

ErrorException: invalid redefinition of constant Node

In [209]:
a2ll(a::Array{}) = let head = Node(a[1], nothing)
    f, l = reduce(a[2:end]; init=(head, head)) do (f, l), x
               l.next = n = Node(x, nothing)
               (f, n)
           end
    l.next = f
end

function Base.getindex(ll::Node{}, i)
    while i != 1
        ll, i = ll.next, i - 1
    end
    ll
end

In [210]:
l2 = 1000000
input2 = @pipe collect("389125467") |> parse.(Int, _) |> vcat(_, (length(_) + 1):l2) |> a2ll

1000000

In [211]:
node_vals = Dict()
head = input2
for i in 1:l2
    node_vals[head.val] = head
    head = head.next
end

In [212]:
function next(cups, current; l=l2)
    cn = node_vals[current]
    r1, r2, r3, after = [cn[x] for x in 2:5]
    cn.next = after
    r3.next = nothing
    
    nc(c) = c < 1 ? nc(l) : c in (r1.val, r2.val, r3.val) ? nc(c - 1) : c
    
    dn = node_vals[nc(current - 1)]
    dn.next, r3.next = r1, dn.next
    
    after.val
end

next (generic function with 1 method)

In [213]:
nrounds(cups; n=100) = reduce((curr, _) -> next(cups, curr), 1:n; init=cups.val)

nrounds (generic function with 1 method)

In [214]:
@time nrounds(input2; n=10_000_000)

 18.877262 seconds (139.39 M allocations: 3.575 GiB, 4.51% gc time)


446131

In [215]:
display(node_vals[1].next.val * node_vals[1].next.next.val)

149245887792